In [1]:
import pandas as pd

train_df = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
train_monthly_df = train_df.groupby(['date_block_num','shop_id','item_id']).agg(item_cnt_month=('item_cnt_day','sum')).reset_index()
train_monthly_df['shop_item'] = train_monthly_df['shop_id'].astype('string') + '-' + train_monthly_df['item_id'].astype('string')
shop_item_list = pd.unique(train_monthly_df['shop_item'])

In [2]:
import numpy as np

# get the sequence of each shop_item
data_seq_dict = {}
for shop_item in shop_item_list:
    data_seq_dict[shop_item] = [0.0 for i in range(0,34)]
for _, row in train_monthly_df.iterrows():
    shop_item = str(row['shop_id']) + '-' + str(row['item_id'])
    data_seq_dict[shop_item][int(row['date_block_num'])] = row['item_cnt_month']

data_seq_array = np.array(list(data_seq_dict.values()))

In [3]:
# get the sequence of cold items
item_seq_dict = {}
item_seq_num_dict = {}
for k, v in data_seq_dict.items():
    item = k.split('-')[1]
    if item not in item_seq_dict.keys():
        item_seq_dict[item] = v
        item_seq_num_dict[item] = 1
    else:
        item_seq_dict[item] = item_seq_dict[item] + v
        item_seq_num_dict[item] += 1
for item in item_seq_dict.keys():
    item_seq_dict[item] = [e / item_seq_num_dict[item] for e in item_seq_dict[item]]

In [ ]:
from sklearn.model_selection import train_test_split

X = data_seq_array[:,0:-1]
Y = data_seq_array[:,1:]
X = X.reshape((X.shape[0], X.shape[1], 1))
Y = Y.reshape((Y.shape[0], Y.shape[1], 1))
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.3)

In [5]:
# # LSTM

# import tensorflow as tf
# from tensorflow.keras import backend as K
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.metrics import RootMeanSquaredError

# def root_mean_squared_error(y_true, y_pred):
#     return K.sqrt(K.mean(K.square(y_pred - y_true))) 

# model = Sequential()
# model.add(LSTM(1, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
# model.compile(loss=root_mean_squared_error, optimizer=Adam(learning_rate=0.1), metrics=[RootMeanSquaredError()])
# model.summary()

# model.fit(x=X_train, y=Y_train, validation_data=(X_val, Y_val), epochs=10, batch_size=1024*256)

In [32]:
# Seq2seq variant 
# 1. step_size = 1 for output
# 2. substitue <EOS> with the last word

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras import Model
from tensorflow.keras import Input
from tensorflow.keras.layers import LSTM, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import RootMeanSquaredError

def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true))) 

input_encoder_vec = Input(name='input', shape = (X_train.shape[1], X_train.shape[2]))
encoder1_vec = LSTM(1, return_sequences=True, name='encoder1')(input_encoder_vec)
encoder2_vec = LSTM(1, return_sequences=True, name='encoder2')(encoder1_vec)
encoder3_vec = LSTM(1, return_sequences=True, name='encoder3')(encoder2_vec)
encoder4_vec, hidden_state_vec, cell_state_vec = LSTM(1, return_sequences=False, return_state=True, name='encoder4')(encoder3_vec)
input_decoder_vec = Lambda(lambda x: tf.slice(x, (0, X_train.shape[1]-1,0), (-1,1,1)))(input_encoder_vec)
decoder_vec = LSTM(1, return_sequences=True, name='decoder')(input_decoder_vec, initial_state=[hidden_state_vec, cell_state_vec])
model = Model(inputs=[input_encoder_vec], outputs=[decoder_vec])
model.compile(loss=root_mean_squared_error, optimizer=Adam(learning_rate=0.1), metrics=[RootMeanSquaredError()])
model.summary()

model.fit(x=X_train, y=Y_train, validation_data=(X_val, Y_val), epochs=10, batch_size=1024*256)

In [37]:
test_df = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv').set_index('ID')
test_df['shop_item'] = test_df['shop_id'].astype('string') + '-' + test_df['item_id'].astype('string')

X_test = []
for _, row in test_df.iterrows():
    shop_item = str(row['shop_id']) + '-' + str(row['item_id'])
    if shop_item in data_seq_dict.keys():
        X_test.append(data_seq_dict[shop_item])
    elif row['item_id'] in item_seq_dict.keys():
        X_test.append(item_seq_dict[row['item_id']])
        print(row['item_id'])
    else:
        X_test.append([0.0 for i in range(0,34)])
X_test = np.asarray(X_test)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
X_test = X_test[:,1:,:]

In [38]:
pred = model.predict(X_test)

pred = pred[:,-1:,:]
pred = pred.reshape((pred.shape[0]))

In [39]:
submission_df = pd.DataFrame(pred, columns=['item_cnt_month'])
submission_df.index.name = 'ID'
submission_df

In [40]:
submission_df.to_csv('/kaggle/working/submission.csv')